In [1]:
#to use it in colab
from google.colab import drive
drive.mount('/content/drive',force_remount=True)
%tensorflow_version 2.x

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:

import tensorflow as tf
import sys
import numpy as np
import pandas as pd
import random
from tensorflow import keras
from matplotlib import pyplot

TensorFlow 2.x selected.


In [ ]:
#Loading Dataset and removing not needed parts
#loading dataset parameters for model

#ratings = pd.read_csv('/content/drive/My Drive/NCF1/1m/ratings.dat',sep='::',names=['UserID','MovieID','Rating','Timestamp'])
ratings = pd.read_csv('/content/drive/My Drive/NCF1/100k/u.data',sep='\t',names=['UserID','MovieID','Rating','Timestamp'])

ratings=ratings[['UserID','MovieID']]
nusers=max(np.unique(ratings['UserID']))
nmovies=max(np.unique(ratings['MovieID']))

In [ ]:
def define_model_NeuMF(num_users,num_items,num_layers,predictivefactors,lr):
  #defining optimizers based on paper
  adamopt=keras.optimizers.Adam()
  sgdopt=keras.optimizers.SGD(learning_rate=lr)

  #Inputs
  Movies = keras.layers.Input(shape=(1,),dtype='int32',name='Movies_Input')
  Users = keras.layers.Input(shape=(1,),dtype='int32',name='Users_Input')
  
  #Movies = keras.utils.to_categorical(Movies,num_classes=num_items+1)
  #Users = keras.utils.to_categorical(Users,num_classes=num_users+1)
  #Movies=tf.one_hot(Movies,depth=num_items+1)
  #Users=tf.one_hot(Users,depth=num_users+1)
  
  #GMF
  gmf_userembed=keras.layers.Embedding(num_users+1,2*predictivefactors,name='GMF_Embed_U')(Users)
  gmf_movieembed=keras.layers.Embedding(num_items+1,2*predictivefactors,name='GMF_Embed_M')(Movies)
  gmf_uservec=keras.layers.Flatten(name='GMF_Flatten_U')(gmf_userembed)
  gmf_movievec=keras.layers.Flatten(name='GMF_Flatten_M')(gmf_movieembed)
  gmf_vec=keras.layers.multiply([gmf_uservec,gmf_movievec],name='GMF_multiply')
  gmf = keras.layers.Dense(1, activation="sigmoid",kernel_initializer=keras.initializers.RandomNormal(mean=0.0, stddev=0.01),name='GMF_Dense')(gmf_vec)
  gmf_model = keras.Model(inputs=[Movies,Users],outputs=gmf)
  gmf_model.compile(optimizer=adamopt,loss='binary_crossentropy',metrics=['accuracy'])
  
  #MLP
  mlp_userembed=keras.layers.Embedding(num_users+1,2*predictivefactors,name='MLP_Embed_U')(Users)
  mlp_movieembed=keras.layers.Embedding(num_items+1,2*predictivefactors,name='MLP_Embed_M')(Movies)
  mlp_uservec=keras.layers.Flatten(name='MLP_Flatten_U')(mlp_userembed)
  mlp_movievec=keras.layers.Flatten(name='MLP_Flatten_M')(mlp_movieembed)
  mlp_vec=keras.layers.concatenate([mlp_uservec,mlp_movievec],name='MLP_Concat')
  mlp=  keras.layers.Dense((2**(num_layers-1))*predictivefactors, activation="relu",kernel_initializer=keras.initializers.RandomNormal(mean=0.0, stddev=0.01),name=('MLP_Dense_'+str(1)))(mlp_vec)
  
  for i in range(num_layers-1):
    mlp = keras.layers.Dense((2**(num_layers-2-i))*predictivefactors, activation="relu",kernel_initializer=keras.initializers.RandomNormal(mean=0.0, stddev=0.01),name=('MLP_Dense_'+str(2+i)))(mlp)
  
  mlp_out=  keras.layers.Dense(1, activation="sigmoid",kernel_initializer=keras.initializers.RandomNormal(mean=0.0, stddev=0.01),name=('MLP_Dense_out'))(mlp)
  mlp_model = keras.Model(inputs=[Movies,Users], outputs=mlp_out)
  mlp_model.compile(optimizer=adamopt,loss='binary_crossentropy',metrics=['accuracy'])

  #NeuMF
  NeuMF_vec = keras.layers.concatenate([mlp, gmf_model.output],name='NeuMF_Concat')
  NeuMF = keras.layers.Dense(1, activation="sigmoid",kernel_initializer=keras.initializers.RandomNormal(mean=0.0, stddev=0.01),name='NeuMF_Dense')(NeuMF_vec)
  # our model will accept the inputs of the two branches and
  # then output a single value
  neumf_model = keras.Model(inputs=[gmf_model.input], outputs=NeuMF)
  neumf_model.compile(optimizer=sgdopt,loss='binary_crossentropy',metrics=['accuracy'])
  return gmf_model,mlp_model,neumf_model

def define_model_NeuMF_pretrained(model1,model2,num_users,num_items,num_layers,predictivefactors,lr,flag=False,model3=None):
  #defining a pretrained neumf based on trained GMF,MLP and optional neumf
  adamopt=keras.optimizers.Adam()
  sgdopt=keras.optimizers.SGD(learning_rate=lr)
  Movies = keras.layers.Input(shape=(1,),dtype='int32',name='Movies_Input')
  Users = keras.layers.Input(shape=(1,),dtype='int32',name='Users_Input')
    
  #GMF
  gmf_userembed=keras.layers.Embedding(num_users+1,2*predictivefactors,name='GMF_Embed_U',weights=model1.layers[2].get_weights())(Users)
  gmf_movieembed=keras.layers.Embedding(num_items+1,2*predictivefactors,name='GMF_Embed_M',weights=model1.layers[3].get_weights())(Movies)
  gmf_uservec=keras.layers.Flatten(name='GMF_Flatten_U')(gmf_userembed)
  gmf_movievec=keras.layers.Flatten(name='GMF_Flatten_M')(gmf_movieembed)
  gmf_vec=keras.layers.multiply([gmf_uservec,gmf_movievec],name='GMF_multiply')
  gmf = keras.layers.Dense(1, activation="sigmoid",name='GMF_Dense',weights=model1.layers[7].get_weights())(gmf_vec)
  
  #MLP
  mlp_userembed=keras.layers.Embedding(num_users+1,2*predictivefactors,name='MLP_Embed_U',weights=model2.layers[2].get_weights())(Users)
  mlp_movieembed=keras.layers.Embedding(num_items+1,2*predictivefactors,name='MLP_Embed_M',weights=model2.layers[3].get_weights())(Movies)
  mlp_uservec=keras.layers.Flatten(name='MLP_Flatten_U')(mlp_userembed)
  mlp_movievec=keras.layers.Flatten(name='MLP_Flatten_M')(mlp_movieembed)
  mlp_vec=keras.layers.concatenate([mlp_uservec,mlp_movievec],name='MLP_Concat')
  mlp=keras.layers.Dense((2**(num_layers-1))*predictivefactors, activation="relu",weights=model2.layers[7].get_weights(),name=('MLP_Dense_'+str(1)))(mlp_vec)
  
  for i in range(num_layers-1):
    mlp = keras.layers.Dense((2**(num_layers-2-i))*predictivefactors, activation="relu",weights=model2.layers[8+i].get_weights(),name=('MLP_Dense_'+str(2+i)))(mlp)

  #NeuMF Layer
  NeuMF_vec = keras.layers.concatenate([mlp, gmf],name='NeuMF_Concat')
  if flag:
    NeuMF = keras.layers.Dense(1, activation="sigmoid",weights=model3.layers[-1].get_weights(),name='NeuMF_Dense')(NeuMF_vec)
  else:
    NeuMF = keras.layers.Dense(1, activation="sigmoid",kernel_initializer=keras.initializers.RandomNormal(mean=0.0, stddev=0.01),name='NeuMF_Dense')(NeuMF_vec)
    
  neumf_model = keras.Model(inputs=[Movies,Users], outputs=NeuMF)
  neumf_model.compile(optimizer=sgdopt,loss='binary_crossentropy',metrics=['accuracy'])
  return neumf_model

  
def summarize_diagnostics(history,a,counter):
	# plot loss
  pyplot.subplot(211)
  pyplot.title('Categorical Cross Entropy Loss')
  pyplot.plot(history.history['loss'], color='blue', label='train')
  pyplot.plot(history.history['val_loss'], color='orange', label='test')
  # plot accuracy
  pyplot.subplot(212)
  pyplot.title('Classification Accuracy')
  pyplot.plot(history.history['accuracy'], color='blue', label='train')
  pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
	# save plot to file
  if a==1:
    filename = '/content/drive/My Drive/NCF/gmf_'+str(counter)
  elif a==2:
    filename = '/content/drive/My Drive/NCF/mlp_'+str(counter)
  elif a==3:
    filename = '/content/drive/My Drive/NCF/neumf_'+str(counter)
  elif a==4:
    filename = '/content/drive/My Drive/NCF/tneumf_'+str(counter)
  pyplot.savefig(filename + '_plot.png')
  pyplot.close()
  return np.nan

def evaluate(df,predvec,k):
  df['pred']=predvec
  hitratio=hr(df,k)
  ndcgscore=ndcg(df,k)
  return hitratio,ndcgscore
def hr(df,k):
  hit=0.0
  for i in range(len(np.unique(df['UserID']))):
    #seprating each user predictions
    _df=df[df['UserID']==i+1]
    _df=_df.sort_values('pred', ascending=False)
    if np.argwhere(_df['label'].values==1).squeeze()<k:
      hit+=1
  hr=hit/len(np.unique(df['UserID']))
  return hr
def ndcg(df,k):
  ndcg=0
  for i in range(len(np.unique(df['UserID']))):
    _df=df[df['UserID']==i+1]
    _df=_df.sort_values('pred', ascending=False)
    loc=np.argwhere(_df['label'].values==1).squeeze()
    if loc<=k:
      ndcg+=(1/np.log2(loc+2))/(1/np.log2(2))    
  ndcg=ndcg/len(np.unique(df['UserID']))
  return ndcg

def preprocess(df,negativeratio):
  random.seed=1
  #creating user item interaction matrix
  pdf=pd.crosstab(df.iloc[:,0],df.iloc[:,1])
  ndf=~pdf+2

  #creating empty dataframes to store needed data
  test=pd.DataFrame()
  validation=pd.DataFrame()
  train=pd.DataFrame()

  for i in range(len(np.unique(df['UserID']))):
    #seprating positive instances
    temp=df[df['UserID']==i+1].iloc[-1,:]
    temp['label']=1
    test=test.append(temp,ignore_index=True)    
    
    temp=df[df['UserID']==i+1].iloc[-2,:]
    temp['label']=1
    validation=validation.append(temp,ignore_index=True)
    
    temp=df[df['UserID']==i+1].iloc[:-2,:]
    temp['label']=1
    train=train.append(temp,ignore_index=True)
    
    #adding negative instances
    if len(np.flatnonzero(ndf.iloc[i,:]))>(negativeratio*len(np.flatnonzero(pdf.iloc[i,:]))+100):
      ids=random.sample(list(np.flatnonzero(ndf.iloc[i,:])),(negativeratio*len(np.flatnonzero(pdf.iloc[i,:]))+100))
    else:
      ids=random.sample(list(np.flatnonzero(ndf.iloc[i,:])),len(list(np.flatnonzero(ndf.iloc[i,:]))))
    
    _a=pd.DataFrame(ids[:negativeratio], columns=['MovieID'])  
    _a['UserID']=i+1
    _a['label']=0
    validation=validation.append(_a,ignore_index=True)
    _a=pd.DataFrame(ids[negativeratio:negativeratio+100], columns=['MovieID'])  
    _a['UserID']=i+1
    _a['label']=0
    test=test.append(_a,ignore_index=True)
    _a=pd.DataFrame(ids[negativeratio+100:], columns=['MovieID'])  
    _a['UserID']=i+1
    _a['label']=0
    train=train.append(_a,ignore_index=True)
  return train.astype('int'),test.astype('int'),validation.astype('int')


In [8]:
train,test,validation=preprocess(ratings,ratio)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


In [ ]:
counter=1

In [ ]:
nlayers=3
pf=32
lrr=.005
bs=1024
ratio=4

gmf,mlp,neumf=define_model_NeuMF(nusers,nmovies,nlayers,pf,lrr)

In [9]:
counter+=1
es = keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
mcgmf = keras.callbacks.ModelCheckpoint('/content/drive/My Drive/NCF/gmf_model'+str(counter)+'.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)
mcmlp = keras.callbacks.ModelCheckpoint('/content/drive/My Drive/NCF/mlp_model'+str(counter)+'.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)
mcneumf = keras.callbacks.ModelCheckpoint('/content/drive/My Drive/NCF/neumf_model'+str(counter)+'.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)
tmcneumf = keras.callbacks.ModelCheckpoint('/content/drive/My Drive/NCF/t_neumf_model'+str(counter)+'.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)

gmf_history = gmf.fit([train.MovieID,train.UserID],y=train.label,epochs=20,shuffle=True,batch_size=bs,verbose=0,
                      validation_data=([validation.MovieID,validation.UserID],validation.label),callbacks=[es,mcgmf])
summarize_diagnostics(gmf_history,1,counter)


mlp_history = mlp.fit([train.MovieID,train.UserID],y=train.label,epochs=20,shuffle=True,batch_size=bs,verbose=0,
                      validation_data=([validation.MovieID,validation.UserID],validation.label),callbacks=[es,mcmlp])
summarize_diagnostics(mlp_history,2,counter)


neumf_history = neumf.fit([train.MovieID,train.UserID],y=train.label,epochs=100,shuffle=True,batch_size=bs,verbose=0,
                          validation_data=([validation.MovieID,validation.UserID],validation.label),callbacks=[es,mcneumf])
summarize_diagnostics(neumf_history,3,counter)



Epoch 00001: val_loss improved from inf to 0.46202, saving model to /content/drive/My Drive/NCF/gmf_model2.h5

Epoch 00002: val_loss improved from 0.46202 to 0.35252, saving model to /content/drive/My Drive/NCF/gmf_model2.h5

Epoch 00003: val_loss improved from 0.35252 to 0.30203, saving model to /content/drive/My Drive/NCF/gmf_model2.h5

Epoch 00004: val_loss improved from 0.30203 to 0.28179, saving model to /content/drive/My Drive/NCF/gmf_model2.h5

Epoch 00005: val_loss improved from 0.28179 to 0.27268, saving model to /content/drive/My Drive/NCF/gmf_model2.h5

Epoch 00006: val_loss improved from 0.27268 to 0.26985, saving model to /content/drive/My Drive/NCF/gmf_model2.h5

Epoch 00007: val_loss did not improve from 0.26985

Epoch 00008: val_loss did not improve from 0.26985

Epoch 00009: val_loss did not improve from 0.26985

Epoch 00010: val_loss did not improve from 0.26985

Epoch 00011: val_loss did not improve from 0.26985

Epoch 00012: val_loss did not improve from 0.26985

E

In [10]:
trained_neumf=define_model_NeuMF_pretrained(gmf,mlp,nusers,nmovies,nlayers,pf,lrr,flag=False,model3=None)
t_neumf_history = trained_neumf.fit([train.MovieID,train.UserID],y=train.label,epochs=100,shuffle=True,batch_size=bs,verbose=0,
                          validation_data=([validation.MovieID,validation.UserID],validation.label),callbacks=[es,tmcneumf])
summarize_diagnostics(t_neumf_history,4,counter)


Epoch 00001: val_loss improved from inf to 0.27532, saving model to /content/drive/My Drive/NCF/t_neumf_model2.h5

Epoch 00002: val_loss did not improve from 0.27532

Epoch 00003: val_loss did not improve from 0.27532

Epoch 00004: val_loss did not improve from 0.27532

Epoch 00005: val_loss did not improve from 0.27532

Epoch 00006: val_loss did not improve from 0.27532

Epoch 00007: val_loss did not improve from 0.27532

Epoch 00008: val_loss did not improve from 0.27532

Epoch 00009: val_loss did not improve from 0.27532

Epoch 00010: val_loss did not improve from 0.27532

Epoch 00011: val_loss did not improve from 0.27532
Epoch 00011: early stopping


In [32]:
print('accuracy:')
print('gmf',sum((gmf.predict([test.MovieID,test.UserID])>.5)==test.label.values.reshape(len(test.label.values),1))/len(test.label.values))
print('mlp:',sum((mlp.predict([test.MovieID,test.UserID])>.5)==test.label.values.reshape(len(test.label.values),1))/len(test.label.values))
print('neumf:',sum((neumf.predict([test.MovieID,test.UserID])>.5)==test.label.values.reshape(len(test.label.values),1))/len(test.label.values))
print('tneumf:',sum((trained_neumf.predict([test.MovieID,test.UserID])>.5)==test.label.values.reshape(len(test.label.values),1))/len(test.label.values))

accuracy:
gmf [0.91439791]
mlp: [0.91055511]
neumf: [0.91781023]
tneumf: [0.91753725]


In [30]:
for tk in range(1,11):
  print('topk - k=',tk)
  print('gmf:',evaluate(test,gmf.predict([test.MovieID,test.UserID]),tk))
  print('mlp:',evaluate(test,mlp.predict([test.MovieID,test.UserID]),tk))
  print('neumf:',evaluate(test,neumf.predict([test.MovieID,test.UserID]),tk))
  print('tneumf:',evaluate(test,trained_neumf.predict([test.MovieID,test.UserID]),tk))


topk - k= 1
gmf: (0.14846235418875928, 0.21403087576882543)
mlp: (0.1569459172852598, 0.23522670325329798)
neumf: (0.1633085896076352, 0.2389131093887318)
tneumf: (0.1569459172852598, 0.2332195036130918)
topk - k= 2
gmf: (0.2523860021208908, 0.25591846855779676)
mlp: (0.28101802757158006, 0.29090008607408285)
neumf: (0.2831389183457052, 0.2898144879677352)
tneumf: (0.27783669141039236, 0.2830604368050325)
topk - k= 3
gmf: (0.3361611876988335, 0.2938253130117649)
mlp: (0.39236479321314954, 0.3260666767121018)
neumf: (0.38494167550371156, 0.3322884221149529)
tneumf: (0.37751855779427357, 0.32599107992157517)
topk - k= 4
gmf: (0.4241781548250265, 0.3131064179322566)
mlp: (0.4740190880169671, 0.3494501443816342)
neumf: (0.4835630965005302, 0.3569025986091975)
tneumf: (0.47720042417815484, 0.3514257289656279)
topk - k= 5
gmf: (0.4740190880169671, 0.3350152374998765)
mlp: (0.5344644750795334, 0.3664483664599599)
neumf: (0.5471898197242842, 0.37238986761389414)
tneumf: (0.542948038176034, 0.3